## Initial Setup

In [1]:
# !pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 51 kB/s 
     |████████████████████████████████| 199 kB 61.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=9ede8936d900f764edd37e8b30991fdc3062754f1c6c5c990e78eaf020af71b5
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1663598779299_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark

In [3]:
# Run this everytime you create a new spark instance. 

spark.sparkContext.install_pypi_package("plotly==5.5.0")
spark.sparkContext.install_pypi_package("pandas==0.25.1")
spark.sparkContext.install_pypi_package("numpy==1.14.5")
spark.sparkContext.install_pypi_package("matplotlib==3.1.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/b3/f4/23d4a698db9fe772f7fdf40ac17b743c4b0d80274732c59db5bd45acb3be/plotly-5.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/a5/f86bc8d67c979020438c8559cc70cfe3a1643fd160d35e09c9cca6a09189/tenacity-8.0.1-py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/3f/e7/7f24ef402a5766c677683e313c5595137d754cb9eb1c99627803280e79d5/numpy-1.14.5-cp37-cp37m-manylinux1_x86_64.whl
  Found existing installation: numpy 1.16.5
    Not uninstalling numpy at /usr/local/lib64/python3.7/site-packages, outside environment /tmp/1663601496015-0

  Using cached https://files

In [4]:
from pyspark.sql.types import StructType, IntegerType, StringType, FloatType,ArrayType
from pyspark.sql.functions import split, col, regexp_replace , round as round2 , coalesce, lit
import pyspark.sql.functions as F
from pyspark.sql.window import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# mounting drive
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


## Defining Custom Functions 

In [5]:
def get_quantiles(df, col_name, quantiles_list = [0.01, 0.25, 0.5, 0.75, 0.99]):
    """
    Takes a numerical column and returns column values at requested quantiles

    Inputs 
    Argument 1: Dataframe
    Argument 2: Name of the column
    Argument 3: A list of quantiles you want to find. Default value [0.01, 0.25, 0.5, 0.75, 0.99]

    Output 
    Returns a dictionary with quantiles as keys and column quantile values as values 
    """
    quantiles_list.append(0)
    quantiles_list.append(1)
    quantiles_list = list(set(quantiles_list))
    quantiles_dict = {}
    # Store min, quantiles and max in output dict, sequentially
    for qnt in quantiles_list:
      quantiles_dict[qnt] = df.select(F.percentile_approx(col_name,qnt,1000000)).collect()[0][0]

    return(quantiles_dict)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def add_OHE_columns (df, n_name_list):
    """
    Given a list of tags, creates one hot encoded columns for each tag. 
  
    Input
    Argument 1: Dataframe in which the function will add the new columns
    Argument 2: list of tags
  
    Output
    Prints the names of columns that have been added 
    Returns the modified dataframe 
    """
    for name in n_name_list:
        df = df.withColumn('has_tag_'+name, F.when(F.array_contains(col('tags'),name)== 'true', 1).otherwise(0))
        print ("added column: has_tag_"+name)

    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read the data

In [7]:
interaction_level_df = spark.read.parquet(
's3://recass/interaction_level_df_post_eda.parquet'
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Adding user level average features

1. user_avg_rating
2. user_n_ratings
3. user_avg_years_betwn_review_and_submission
4. user_avg_prep_time_recipes_reviewed
5. user_avg_n_steps_recipes_reviewed
6. user_avg_n_ingredients_recipes_reviewed

In [8]:
windowSpec = Window.partitionBy('user_id')
interaction_level_df = interaction_level_df.withColumn('user_avg_rating',F.avg(col('rating')).over(windowSpec))\
.withColumn('user_n_ratings',F.count(col('rating')).over(windowSpec))\
.withColumn('user_avg_years_betwn_review_and_submission',F.avg(col('years_since_submission_on_review_date')).over(windowSpec))\
.withColumn('user_avg_prep_time_recipes_reviewed',F.avg(col('minutes')).over(windowSpec))\
.withColumn('user_avg_n_steps_recipes_reviewed',F.avg(col('n_steps')).over(windowSpec))\
.withColumn('user_avg_n_ingredients_recipes_reviewed',F.avg(col('n_ingredients')).over(windowSpec)) 


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Add columns 

7. user_avg_calories

8. user_avg_total_fat_per_100_cal_recipes_reviewed

9. user_avg_sugar_per_100_cal_recipes_reviewed

10. user_avg_sodium_per_100_cal_recipes_reviewed

11. user_avg_protein_per_100_cal_recipes_reviewed

12. user_avg_saturated_fat_per_100_cal_recipes_reviewed

13. user_avg_carbohydrates_per_100_cal_recipes_reviewed

In [9]:
interaction_level_df = interaction_level_df.withColumn('user_avg_calories',F.avg(col('calories')).over(windowSpec))\
.withColumn('user_avg_total_fat_per_100_cal_recipes_reviewed',F.avg(col('total_fat_per_100_cal')).over(windowSpec)) \
.withColumn('user_avg_sugar_per_100_cal_recipes_reviewed',F.avg(col('sugar_per_100_cal')).over(windowSpec)) \
.withColumn('user_avg_sodium_per_100_cal_recipes_reviewed',F.avg(col('sodium_per_100_cal')).over(windowSpec)) \
.withColumn('user_avg_protein_per_100_cal_recipes_reviewed',F.avg(col('protein_per_100_cal')).over(windowSpec)) \
.withColumn('user_avg_saturated_fat_per_100_cal_recipes_reviewed',F.avg(col('saturated_fat_per_100_cal')).over(windowSpec)) \
.withColumn('user_avg_carbohydrates_per_100_cal_recipes_reviewed',F.avg(col('carbohydrates_per_100_cal')).over(windowSpec))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Code check cell
assert(round(interaction_level_df.filter('user_id == 601529').select('user_avg_rating').first()[0], 2) == 4.22)
assert(interaction_level_df.filter('user_id == 601529').select('user_n_ratings').first()[0] == 27)
assert(round(interaction_level_df.filter('user_id == 601529').select('user_avg_years_betwn_review_and_submission').first()[0], 2) == 3.51)
assert(interaction_level_df.filter('user_id == 233044').select('user_avg_prep_time_recipes_reviewed').first()[0] == 50.3)
assert(interaction_level_df.filter('user_id == 233044').select('user_avg_n_steps_recipes_reviewed').first()[0] == 8.8)
assert(interaction_level_df.filter('user_id == 233044').select('user_avg_n_ingredients_recipes_reviewed').first()[0] == 8.2)
assert(round(interaction_level_df.filter('user_id == 233044').select('user_avg_total_fat_per_100_cal_recipes_reviewed').first()[0]) == 6)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**More Features:**

high_ratings = 5 rating

- `user_avg_years_betwn_review_and_submission_high_ratings`
- `user_avg_prep_time_recipes_reviewed_high_ratings`
- `user_avg_n_steps_recipes_reviewed_high_ratings`
- `user_avg_n_ingredients_recipes_reviewed_high_ratings`

In [11]:
# Create indicator columns for highest ratings 
interaction_level_df.createOrReplaceTempView('interaction_level_df2')
interaction_level_df2 = spark.sql("SELECT user_id,recipe_id, rating, review, review_date, name, id, minutes, contributor_id, submitted, \
tags, nutrition, n_steps, steps, description, ingredients, n_ingredients, calories, total_fat_PDV, sugar_PDV, sodium_PDV, \
protein_PDV, saturated_fat_PDV, carbohydrates_PDV, total_fat_per_100_cal, sugar_per_100_cal, sodium_per_100_cal, protein_per_100_cal, \
saturated_fat_per_100_cal, carbohydrates_per_100_cal, days_since_submission_on_review_date, months_since_submission_on_review_date, \
years_since_submission_on_review_date, years_since_submission_on_review_date_bucket, prep_time_bucket, n_steps_bucket, \
n_ingredients_bucket, calories_bucket, total_fat_PDV_bucket, sugar_PDV_bucket, sodium_PDV_bucket, protein_PDV_bucket, saturated_fat_PDV_bucket, \
carbohydrates_PDV_bucket, total_fat_per_100_cal_bucket, sugar_per_100_cal_bucket, sodium_per_100_cal_bucket, protein_per_100_cal_bucket, \
saturated_fat_per_100_cal_bucket, carbohydrates_per_100_cal_bucket, user_avg_rating, user_n_ratings, user_avg_years_betwn_review_and_submission, \
user_avg_prep_time_recipes_reviewed, user_avg_n_steps_recipes_reviewed, user_avg_n_ingredients_recipes_reviewed, user_avg_calories, \
user_avg_total_fat_per_100_cal_recipes_reviewed, user_avg_sugar_per_100_cal_recipes_reviewed, user_avg_sodium_per_100_cal_recipes_reviewed, \
user_avg_protein_per_100_cal_recipes_reviewed, user_avg_saturated_fat_per_100_cal_recipes_reviewed, user_avg_carbohydrates_per_100_cal_recipes_reviewed \
FROM interaction_level_df2 ; " )
interaction_level_df2.createOrReplaceTempView('interaction_level_df3')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
interaction_level_df = spark.sql("SELECT *,\
case when rating =5 then 1 else 0 end as highest_rating_flag, \
sum(case when rating =5 then 1 else 0 end  ) over(partition by user_id) as user_n_5_ratings,\
avg(case when rating =5 then years_since_submission_on_review_date end ) over(partition by user_id) as user_avg_years_betwn_review_and_submission_5_ratings, \
avg(case when rating =5 then minutes end ) over(partition by user_id) as user_avg_prep_time_recipes_reviewed_5_ratings, \
avg(case when rating =5 then n_steps end ) over(partition by user_id) as user_avg_n_steps_recipes_reviewed_5_ratings, \
avg(case when rating =5 then n_ingredients end ) over(partition by user_id) as user_avg_n_ingredients_recipes_reviewed_5_ratings, \
avg(case when rating =5 then calories end ) over(partition by user_id) as  user_avg_calories_recipes_reviewed_5_ratings,\
avg(case when rating =5 then sugar_per_100_cal end ) over(partition by user_id) as  user_avg_sodium_per_100_cal_recipes_reviewed_5_ratings,\
avg(case when rating =5 then protein_per_100_cal end ) over(partition by user_id) as  user_avg_protein_per_100_cal_recipes_reviewed_5_ratings,\
avg(case when rating =5 then saturated_fat_per_100_cal end ) over(partition by user_id) as  user_avg_saturated_fat_per_100_cal_recipes_reviewed_5_ratings,\
avg(case when rating =5 then total_fat_per_100_cal end ) over(partition by user_id) as  user_avg_total_fat_per_100_cal_recipes_reviewed_5_ratings,\
avg(case when rating =5 then carbohydrates_per_100_cal end ) over(partition by user_id) as  user_avg_carbohydrates_per_100_cal_recipes_reviewed_5_ratings  FROM interaction_level_df3;")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# leaving null values as it is , because it is an indicator of no rating being given as 5 stars

In [13]:
assert(interaction_level_df.filter('user_id == 233044').select('user_n_5_ratings').first()[0] == 7)
assert(round(interaction_level_df.filter('user_id == 233044').select('user_avg_years_betwn_review_and_submission_5_ratings').first()[0], 2) == 2.24)
assert(round(interaction_level_df.filter('user_id == 233044').select('user_avg_prep_time_recipes_reviewed_5_ratings').first()[0]) == 46)
assert(round(interaction_level_df.filter('user_id == 233044').select('user_avg_n_steps_recipes_reviewed_5_ratings').first()[0], 2) == 7.29)
assert(round(interaction_level_df.filter('user_id == 233044').select('user_avg_n_ingredients_recipes_reviewed_5_ratings').first()[0], 2) == 6.86)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
interaction_level_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: date (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- saturated_fat_PDV: float (nullable = true)
 |-- carb

## Tags level EDA

In [15]:
interaction_tag_level_df = interaction_level_df.withColumn('individual_tag',F.explode('tags'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
tags_ratings_summary = (interaction_tag_level_df
                        .groupBy('individual_tag').agg(F.avg('rating').alias('avg_user_rating'),
#                                                      F.max('rating').alias('max_user_rating'),
#                                                      F.min('rating').alias('min_user_rating'),
                                                       F.count('rating').alias('n_user_ratings'),
                                                       F.countDistinct('id').alias('n_recipes')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
interactions, recipes  =  interaction_level_df.count(), interaction_level_df.agg(F.countDistinct('id')).first()[0]

tags_ratings_summary = (tags_ratings_summary.withColumn("in_percent_recipies", F.col ("n_recipes")/F.lit(recipes))
                                            .withColumn("in_percent_interactions", F.col ("n_user_ratings")/F.lit(interactions)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 1. Top ```n``` most rated tags

In [18]:
tags_ratings_summary.sort(col('n_user_ratings').desc()).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------+---------+-------------------+-----------------------+
|    individual_tag|   avg_user_rating|n_user_ratings|n_recipes|in_percent_recipies|in_percent_interactions|
+------------------+------------------+--------------+---------+-------------------+-----------------------+
|       preparation|4.4119124813277715|       1123326|   229318| 0.9952779007491125|     0.9970859455232471|
|      time-to-make| 4.414416558383976|       1105132|   224098| 0.9726222407402585|       0.98093659823417|
|            course| 4.412402044928726|       1071920|   217130| 0.9423799727437654|     0.9514569828574067|
|           dietary| 4.412032038984685|        901277|   163918| 0.7114311259255401|     0.7999909462821618|
|   main-ingredient| 4.424040070642098|        864074|   169549| 0.7358705936477349|     0.7669688418963456|
|              easy|4.4183637556952755|        630786|   125789| 0.5459449840715953|     0.5598978882646952|
|          occasion

Drop tags present in majority of recipes more than 75% rows 

In [19]:
tags_ratings_summary_raw = tags_ratings_summary

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
tags_ratings_summary = tags_ratings_summary_raw.filter('in_percent_interactions<=0.75')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# print the quantiles of in_percent_interactions  
get_quantiles(df = tags_ratings_summary , 
              col_name = 'in_percent_interactions' , 
              quantiles_list = [0.01,0.25,0.5, 0.75,0.8,0.85,0.9,0.95, 0.99])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.25: 0.00035948585534111656, 0.5: 0.0034235480100016954, 0.8: 0.02746826982564492, 0.75: 0.018886765506045133, 0.85: 0.04430729738533955, 0.9: 0.08068815356525645, 0.95: 0.16414301678754564, 0.99: 0.34091596995940915, 0: 8.876193959039915e-07, 1: 0.5598978882646952, 0.01: 8.876193959039915e-07}

In [22]:
# keeping the tags appearing in the top 5 percentile 
top_most_frequent_tags = tags_ratings_summary.filter('in_percent_interactions>=0.16')

top_most_frequent_tags.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

29

In [23]:
top_frequent_tags_list = list(top_most_frequent_tags.toPandas()['individual_tag'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
interaction_level_df = add_OHE_columns(interaction_level_df,top_frequent_tags_list)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

added column: has_tag_desserts
added column: has_tag_inexpensive
added column: has_tag_main-dish
added column: has_tag_beginner-cook
added column: has_tag_low-cholesterol
added column: has_tag_taste-mood
added column: has_tag_oven
added column: has_tag_dinner-party
added column: has_tag_4-hours-or-less
added column: has_tag_number-of-servings
added column: has_tag_equipment
added column: has_tag_30-minutes-or-less
added column: has_tag_cuisine
added column: has_tag_meat
added column: has_tag_occasion
added column: has_tag_north-american
added column: has_tag_kid-friendly
added column: has_tag_15-minutes-or-less
added column: has_tag_comfort-food
added column: has_tag_vegetables
added column: has_tag_low-in-something
added column: has_tag_healthy
added column: has_tag_low-calorie
added column: has_tag_american
added column: has_tag_holiday-event
added column: has_tag_easy
added column: has_tag_low-sodium
added column: has_tag_low-carb
added column: has_tag_60-minutes-or-less

#### 2.  Bottom ```n``` least rated tags

In [25]:
tags_ratings_summary.sort(col('in_percent_interactions')).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+--------------+---------+--------------------+-----------------------+
|    individual_tag|avg_user_rating|n_user_ratings|n_recipes| in_percent_recipies|in_percent_interactions|
+------------------+---------------+--------------+---------+--------------------+-----------------------+
|         pork-loin|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|breakfast-potatoes|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|   black-bean-soup|            3.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|     desserts-easy|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|     chicken-stews|            4.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
+------------------+---------------+--------------+---------+--------------------+-----------------------+
only showing top 5 rows

The above tags are present in 1 recipe in over two hundred thousand. The features we create based on these tags will not teach the model new information. If these tags were one hot encoded, the entire column would be filled with zeros, and only a few rows will have 1s. One hot encoding of these tags is not a good idea. If you come up with an encoding that captures the rarity of these tags, only then can you add these tags to the analysis.

#### 3. Top ```n``` rated tags 

In [26]:
tags_ratings_summary.sort(col('avg_user_rating').desc()).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+--------------+---------+--------------------+-----------------------+
|      individual_tag|avg_user_rating|n_user_ratings|n_recipes| in_percent_recipies|in_percent_interactions|
+--------------------+---------------+--------------+---------+--------------------+-----------------------+
|  breakfast-potatoes|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|           pork-loin|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|middle-eastern-ma...|            5.0|             2|        1|4.340164752654011E-6|   1.775238791807983E-6|
|Throwtheultimatef...|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
|       desserts-easy|            5.0|             1|        1|4.340164752654011E-6|   8.876193959039915E-7|
+--------------------+---------------+--------------+---------+--------------------+-----------------------+
only showing top 5 

Top rated tags have low number of ratings.

In [27]:
get_quantiles (df = tags_ratings_summary, 
               col_name ='n_user_ratings', 
               quantiles_list = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.5, 0.75, 0.99])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.05: 1, 0.1: 12, 0.2: 187, 0.15: 99, 0.25: 405, 0.5: 3857, 0.75: 21278, 0.99: 384079, 0: 1, 1: 630786, 0.01: 1}

In [28]:
tags_ratings_summary_raw2 = tags_ratings_summary

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Dropping the tags with less than 100 ratings 
tags_ratings_summary = tags_ratings_summary.filter('n_user_ratings >=100')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
get_quantiles(df = tags_ratings_summary, 
              col_name = 'avg_user_rating',
              quantiles_list = [0.01,0.25,0.5, 0.75,0.8,0.85,0.9,0.95, 0.99])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.25: 4.344224006718102, 0.5: 4.403587357364895, 0.8: 4.461777662606893, 0.75: 4.451890903550206, 0.85: 4.48284199758207, 0.9: 4.503597122302159, 0.95: 4.536699941280094, 0.99: 4.609756097560975, 0: 3.6742424242424243, 1: 4.822727272727272, 0.01: 3.962536023054755}

In [31]:
# keeping tags above 95 percentile
top_rated_tags_df = tags_ratings_summary.filter('avg_user_rating>=4.53')

top_rated_tags_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

26

In [32]:
top_rated_tags_list = list(top_rated_tags_df.toPandas()['individual_tag'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Check if any of the current tags have been added earlier

In [33]:
set(top_frequent_tags_list) & set(top_rated_tags_list)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

set()

In [34]:
all_added_columns_set = set(top_frequent_tags_list).union(set(top_rated_tags_list))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
interaction_level_df = add_OHE_columns(interaction_level_df,top_rated_tags_list)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

added column: has_tag_beverages
added column: has_tag_ragu-recipe-contest
added column: has_tag_avocado
added column: has_tag_simply-potatoes2
added column: has_tag_asparagus
added column: has_tag_turkey-burgers
added column: has_tag_greek
added column: has_tag_omelets-and-frittatas
added column: has_tag_strawberries
added column: has_tag_smoothies
added column: has_tag_grilling
added column: has_tag_labor-day
added column: has_tag_shakes
added column: has_tag_mashed-potatoes
added column: has_tag_non-alcoholic
added column: has_tag_salads
added column: has_tag_australian
added column: has_tag_a1-sauce
added column: has_tag_lettuces
added column: has_tag_mango
added column: has_tag_polynesian
added column: has_tag_salsas
added column: has_tag_barbecue
added column: has_tag_punch
added column: has_tag_cocktails
added column: has_tag_memorial-day

#### 3. Bottom ```n``` rated tags 

In [36]:
get_quantiles (df = tags_ratings_summary, 
               col_name ='n_user_ratings', 
               quantiles_list = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.5, 0.75, 0.99])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0.05: 180, 0.1: 330, 0.2: 806, 0.15: 522, 0.25: 1143, 0.5: 6312, 0.75: 27839, 0.99: 445959, 0: 104, 1: 630786, 0.01: 114}

In [37]:
# checking for all cases within bottom 1 percentile
tags_ratings_summary.filter('avg_user_rating<=4').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6

In [38]:
bottom_rated_tags_list = list(tags_ratings_summary.filter('avg_user_rating<=4').toPandas()['individual_tag'])
bottom_rated_tags_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['jellies', 'water-bath', 'birthday', 'honduran', 'pressure-canning', 'unprocessed-freezer']

Check if any of the current tags have been added earlier

In [39]:
all_added_columns_set & set(bottom_rated_tags_list)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

set()

In [40]:
interaction_level_df = add_OHE_columns(interaction_level_df,bottom_rated_tags_list)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

added column: has_tag_jellies
added column: has_tag_water-bath
added column: has_tag_birthday
added column: has_tag_honduran
added column: has_tag_pressure-canning
added column: has_tag_unprocessed-freezer

## Final DataFrame

In [41]:
len(interaction_level_df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

136

In [42]:
interaction_level_df.write.mode('overwrite').parquet('s3://recass/interaction_level_df_post_part2_feature_extraction.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…